In [1]:
from splink import DuckDBAPI, block_on
from splink.blocking_analysis import count_comparisons_from_blocking_rule
from splink import Linker

import pandas as pd
import json

## Importing Datasets

In [2]:
df_dbusa = pd.read_csv("dbusa_presentation_202410302126.csv",
                       dtype={"zip_code" : str})
df_reonomy = pd.read_csv("reonomy_presentation_202410302125.csv",
                       dtype={"zip_code" : str})

## Loading the model

In [3]:
with open("state_only_blocking_model.json") as f:
  settings = json.load(f)

linker = Linker([df_dbusa , df_reonomy], settings, db_api=DuckDBAPI(connection=":temporary:"))

### Creating Predictions

In [4]:
df_predictions = linker.inference.predict(threshold_match_probability=0.2)

Blocking time: 90.53 seconds
Predict time: 507.39 seconds


## Start Combining Similar Clusters

In [21]:
clusters = linker.clustering.cluster_pairwise_predictions_at_threshold(
    df_predictions, threshold_match_probability=0.05
)

Completed iteration 1, num representatives needing updating: 11974
Completed iteration 2, num representatives needing updating: 162
Completed iteration 3, num representatives needing updating: 0


In [27]:
clusters_df = clusters.as_pandas_dataframe()

In [ ]:
"""
Check the number of the clusters and it's 757 clusters
"""
clusters_df["cluster_id"].nunique()

757

## Top 10 Clusters by size

In [29]:
cluster_by_size = clusters_df.groupby("cluster_id")\
                    .size().sort_values(ascending=False)\
                    .reset_index(name="cluster_size")



In [30]:
cluster_by_size.head(10)

,cluster_id,cluster_size
0,__splink__input_table_0-__-11132541471303,17431
1,__splink__input_table_0-__-11132541477159,13204
2,__splink__input_table_0-__-11132541469754,10668
3,__splink__input_table_0-__-11132541473668,10506
4,__splink__input_table_0-__-11132541470472,6892
5,__splink__input_table_0-__-11132541473280,5411
6,__splink__input_table_0-__-11132541476336,5300
7,__splink__input_table_0-__-11132541472391,4619
8,__splink__input_table_0-__-11132541473850,4323
9,__splink__input_table_0-__-11132541470534,4211


In [37]:
clusters_df[clusters_df["cluster_id"] == "__splink__input_table_0-__-11132541471303"].sample(10)

,cluster_id,property_id,state,county,city,address,zip_code,source_dataset
35879,__splink__input_table_0-__-11132541471303,11132549603855,CA,orange,santa ana,650 s grand ave ste 104,92705,dbusa
38513,__splink__input_table_0-__-11132541471303,11140024408838,CA,san diego,la mesa,8750 mellmanor dr apt 25,91942,dbusa
34195,__splink__input_table_0-__-11132541471303,11132543947198,CA,nevada,nevada,201 church st ste 1,95959,dbusa
34835,__splink__input_table_0-__-11132541471303,11140009558419,CA,orange,irvine,18881 von karman ave ste 920,92612,dbusa
40226,__splink__input_table_0-__-11132541471303,11132549846949,CA,san joaquin,stockton,2300 pacific ave,95204,dbusa
4848,__splink__input_table_0-__-11132541471303,dbcdd644-b853-59f0-bdda-d486b06ac6d3,CA,sonoma,petaluma,1301 southpoint blvd,94954,reonomy
28780,__splink__input_table_0-__-11132541471303,11132550528150,CA,fresno,fresno,5711 n 1st st,93710,dbusa
33229,__splink__input_table_0-__-11132541471303,11132574077579,CA,los angeles,tarzana,18321 ventura blvd ste 780,91356,dbusa
30660,__splink__input_table_0-__-11132541471303,11140031643902,CA,los angeles,long beach,4720 e 2nd st ste 2,90803,dbusa
12795,__splink__input_table_0-__-11132541471303,1d7020ff-35a9-5bdb-be73-835d9871d164,CA,san luis obispo,atascadero,6955 el camino real,93422,reonomy


After sampling from the dataset we can find that the biggest cluster by size consist of counties and cities in Californaia state so the model is perfectly blocked in state 

## Final Editiing in the clusers dataframe 

In [31]:
clusters_df["source_dataset"] = clusters_df["source_dataset"].replace({
    "__splink__input_table_0":"dbusa",
    '__splink__input_table_1' : "reonomy"

})

In [38]:
clusters_df.to_csv("usa_peoperties.csv" , index=False , header=True)